### Import

#### cv2 설치하기

In [1]:
import random 
import pandas as pd 
import numpy as np
import os 
import cv2 

#### random: 랜덤 관련한 함수들을 모아놓은 모듈
#### pandas: 데이터분석 라이브러리, 행과 열로 이루어진 데이터 객체를 만들어서 다룸.
#### numpy: 과학 계산을 위한 라이브러리로서 다차원 배열을 처리하는데 필요한 여러 유용한 기능을 제공
#### os: Operating System의 약자로서 운영체제에서 제공되는 여러 기능을 파이썬에서 수행
#### cv2: 실시간 영상 처리에 중점을 둔 영상 처리 라이브러리

In [2]:
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

#### sklearn-preprocessing: 스케일링 및 변수변환을 위한 StandardScaler 클래스는 기능을 제공
#### sklearn-countvectorizer: 문서를 token count matrix로 변환하는 클래스
#### sklearn-train_test_split: 손쉽게 train set(학습 데이터 셋)과 test set(테스트 셋)을 분리

#### torch 모듈 설치

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

#### torch: 오픈소스 머신 러닝 라이브러리, 자동 미분 모듈, 최적화 모듈, 이미지 처리 모듈, 오디오 처리 모듈 등 머신 러닝 및 딥러닝을 위한 다양한 모듈 제공
#### torch-nn: 신경망 모델 정의 (클래스)
#### torch-optim: 다양한 최적화 알고리즈을 구현한 패키지, 모델 매개변수 최적화
#### torch-F: 신경망 모델 정의 (함수)
#### torch-Dataset, DataLoader: Dataset 은 샘플과 정답(label)을 저장하고, DataLoader 는 Dataset 을 샘플에 쉽게 접근할 수 있도록 순회 가능한 객체(iterable)로 감쌉니다.

In [4]:
from tqdm.auto import tqdm

#### tqdm: 반복문이 어디까지 진행되었는지 표시

#### albumentations 패키지 설치

In [5]:
import albumentations as A 
import albumentations
import albumentations.pytorch
from albumentations.pytorch.transforms import ToTensorV2 
import torchvision.models as models

#### albumentations: 이미지를 손쉽게 aygmentation 해주는 파이썬 라이브러리이다.

In [6]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report

#### sklearn의 분류 성능평가

In [7]:
import warnings
warnings.filterwarnings(action='ignore')

#### warnings: 경고 메세지 무시하기

In [8]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#### gpu 사용하기 위한 코드

### Hyperparameter Setting

In [9]:
CFG = {
    'IMG_SIZE':128,
    'EPOCHS':5,
    'LEARNING_RATE':0.01,
    'BATCH_SIZE':32,
    'SEED':41
}

#### 이미지 사이즈, 이폭, 학습률, 배치사이즈, 시드 고정
#### epoch: 훈련 데이터셋에 포함된 모든 데이터들이 한 번씩 모댈을 통과한 횟수로, 모든 학습 데이터셋을 학습하는 횟수를 의미한다. 
#### epoch가 5회라면, 학습 데이터 셋을 5회 모델에 학습시켰다는 것이다.
#### epoch를 높일수록, 다양한 무작위 가중치를 학습해보므로, 적합한 파라미터를 찾을 확률이 올라간다. 하지만 지나치게 높이게 되면, 과적합 현상이 일어날 수 있다.

#### batch size는 연산 한 번에 들어가는 데이터의 크기를 가리킨다.
#### 배치 사이즈가 너무 큰 경우 한 번에 처리해야 할 데이터의 양이 많아지므로, 학습 속도가 느려지고, 메모리 부족 문제가 발생할 위험이 있다.
#### 반대로 배치 사이즈가 너무 작은 경우 적은 데이터를 대상으로 가중치를 업데이트하고, 이 업데이트가 자주발생하므로, 훈련이 불안정해진다.

### Fixed RandomSeed

In [10]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

### Data Load & Train/Validation Split

In [11]:
all_df = pd.read_csv('C:/Users/moond/open/Data/train.csv', encoding='cp949')

In [12]:
train_df, val_df, _, _ = train_test_split(all_df, all_df['cat3'], test_size=0.2, random_state=CFG['SEED'])

#### train set, validation set 구별

### Label-Encoding

In [13]:
le = preprocessing.LabelEncoder()
le.fit(train_df['cat3'].values)

LabelEncoder()

#### 카테고리형 데이터를 수치형으로 변환하는 labelencoder

In [14]:
train_df['cat3'] = le.transform(train_df['cat3'].values)
val_df['cat3'] = le.transform(val_df['cat3'].values)

#### cat3에 labelencoder를 적용하기

### Vectorizer

In [15]:
vectorizer = CountVectorizer(max_features=4096)

#### overview를 vectorize하는 vectorizer 선언, 최대 특성 수는 4096

In [16]:
train_vectors = vectorizer.fit_transform(train_df['overview'])
train_vectors = train_vectors.todense()

val_vectors = vectorizer.transform(val_df['overview'])
val_vectors = val_vectors.todense()

In [17]:
train_vectors.shape

(13588, 4096)

In [18]:
val_vectors.shape

(3398, 4096)

### 이미지 데이터 전처리

#### albumentations 라이브러리를 이용하여 이미지 데이터 다루기

#### albumentation으로 데이터를 커스터마이징하면 훨씬 자유도가 높고 속도가 빠르지만, pytorch data pipeine까지 직접 커스텀해야함.

#### 1. data pipeline 만들기

In [20]:
class ImageDataset(Dataset):
    def __init__(self, img_path_list, text_vectors, label_list, transforms, infer=False):
        self.img_path_list = img_path_list
        self.text_vectors = text_vectors
        self.label_list = label_list
        self.transforms = transforms
        self.infer = infer
        
    def __getitem__(self, index):
    
        img_path = self.img_path_list[index] # 이미지 읽기
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image'] # transforms 적용
            
            
    def __len__(self):
        return len(self.img_path_list)  

#### __init__()은 반드시 첫 번째 인수로 self를 지정해야한다. self에는 인스턴스 자체가 전달되어 있다. 이로 인해, 최과 메소드 내에 인스턴스 변수를 작성하거나, 참고하는 것이 가능해진다. 
#### tranform()은 특질들을 추출하는 함수로 기존에 있던 특질로부터 새로운 특질을 추출할 떄 사용한다.
#### 파이프라인을 사용하면 데이터 전처리와 모델 학습, 예측까지 한번에 가능하여 코드도 간결해지는 장점이 있다.

In [ ]:
        # Label
        if self.infer: # infer == True, test_data로부터 label "결과 추출" 시 사용
            return image, torch.Tensor(text_vector).view(-1)
        else: # infer == False
            label = self.label_list[index] # dataframe에서 label 가져와 "학습" 시 사용
            return image, torch.Tensor(text_vector).view(-1), label

#### 2. data augmentation 적용하기 (image를 resize한 뒤에 normalize를 적용)

In [20]:
from torchvision import transforms
import torchvision
import os

In [21]:
import os
print(os.getcwd())

C:\Users\moond\open\Code


In [22]:
# 경로 지정

train_path = 'C:/Users/moond/open/Code/train2'
test_path = 'C:/Users/moond/open/Code/test2'

In [23]:
resize_trans = transforms.Compose([
                                   transforms.Resize((128,128)),
                                   transforms.ToTensor()
])

In [24]:
resize_train = torchvision.datasets.ImageFolder(root=train_path, transform=resize_trans)
resize_test = torchvision.datasets.ImageFolder(root=test_path, transform=resize_trans)

In [25]:
resize_train[0][0]

tensor([[[0.3137, 0.3098, 0.3020,  ..., 0.1490, 0.1529, 0.1529],
         [0.3137, 0.3098, 0.3059,  ..., 0.1529, 0.1529, 0.1529],
         [0.3137, 0.3098, 0.3059,  ..., 0.1569, 0.1529, 0.1529],
         ...,
         [0.4980, 0.4863, 0.4824,  ..., 0.3490, 0.3569, 0.3490],
         [0.4784, 0.4078, 0.3647,  ..., 0.3490, 0.3569, 0.3529],
         [0.3294, 0.3137, 0.3098,  ..., 0.3608, 0.3569, 0.3569]],

        [[0.3412, 0.3373, 0.3333,  ..., 0.2196, 0.2196, 0.2196],
         [0.3412, 0.3373, 0.3373,  ..., 0.2275, 0.2235, 0.2235],
         [0.3412, 0.3373, 0.3373,  ..., 0.2275, 0.2235, 0.2275],
         ...,
         [0.4314, 0.4196, 0.4118,  ..., 0.2980, 0.2980, 0.2902],
         [0.4157, 0.3451, 0.3059,  ..., 0.2980, 0.3059, 0.2980],
         [0.2784, 0.2706, 0.2667,  ..., 0.3020, 0.3020, 0.2980]],

        [[0.4118, 0.4157, 0.4157,  ..., 0.3608, 0.3608, 0.3529],
         [0.4157, 0.4118, 0.4157,  ..., 0.3608, 0.3569, 0.3529],
         [0.4157, 0.4118, 0.4118,  ..., 0.3608, 0.3608, 0.

In [26]:
import numpy as np
np.mean(resize_train[0][0].numpy(),axis=(1,2)) 

array([0.36114624, 0.3708664 , 0.379108  ], dtype=float32)

In [27]:
np.mean(resize_test[0][0].numpy(),axis=(1,2))

array([0.41174626, 0.37635332, 0.38942945], dtype=float32)

In [28]:
def get_mean_std(dataset):
  meanRGB = [np.mean(image.numpy(), axis=(1,2)) for image,_ in dataset]
  stdRGB = [np.std(image.numpy(), axis=(1,2)) for image,_ in dataset]

  meanR = np.mean([m[0] for m in meanRGB])
  meanG = np.mean([m[1] for m in meanRGB])
  meanB = np.mean([m[2] for m in meanRGB])

  stdR = np.mean([s[0] for s in stdRGB])
  stdG = np.mean([s[1] for s in stdRGB])
  stdB = np.mean([s[2] for s in stdRGB])

  print(meanR, meanG, meanB)
  print(stdR, stdG, stdB)

In [30]:
get_mean_std(resize_train)

0.49876145 0.49929118 0.4725662
0.23629072 0.23496795 0.26324624


In [31]:
get_mean_std(resize_test)

0.5003475 0.500986 0.4740023
0.23637316 0.23554333 0.2639925


In [21]:
train_transform = A.Compose([
A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
A.Normalize(mean=(0.499, 0.499, 0.473), std=(0.236, 0.235, 0.263), max_pixel_value=255.0, always_apply=False, p=1.0),
ToTensorV2()
])

test_transform = A.Compose([
A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
A.Normalize(mean=(0.5, 0.5, 0.474), std=(0.236, 0.236, 0.264), max_pixel_value=255.0, always_apply=False, p=1.0),
ToTensorV2()
])

#### 이미지 데이터 normalization
#### transform = A.Compose([])을 이용하여 이미지와 라벨 각각에 Augmentation을 적용하기 위한 객체를 생성
#### 128x128 size로 resize
#### Normalize() -> 입력 받은 이미지 값의 범위를 (0, 255) → (-1, 1) 범위로 줄여주는 역할
#### ToTensorV2 -> tensor형 변환 (tensor: 딥러닝에서 데이터를 처리하기 위한 데이터의 형태)

#### 3. albumentation으로 data augmentation해주기

In [31]:
# __init__(self, img_path_list, text_vectors, label_list, transforms, infer=False)
train_dataset = ImageDataset(train_df['img_path'].values, train_vectors, train_df['cat3'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0) 

val_dataset = ImageDataset(val_df['img_path'].values, val_vectors, val_df['cat3'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

#### Dataloader() -> 모델에 데이터를 넣어주기 전, 적당한 양씩 데이터를 나누는 과정이 필요하게 되고 dataloader에서 이 과정을 수월하게 만들어주는 기능을 한다.
#### Dataset() -> 데이터의 집합을 의미
#### shuffle() -> 리스트를 무작위로 셔플하기, 순서 섞기
#### num_workers -> 학습 도충 CPU의 작업을 몇 개의 코어를 사용해서 진행할지에 대한 설정 파라미터입니다. #### 하이퍼파라미터를 튜닝하는 것처럼 모델에 가장 적합한 num_workers 수치를 찾아내는 것도 파라미터 튜닝이다.

#### 이미지의 픽셀 0~255, 이것의 중간값인 128x128 크기로 변환/ train test 이미지 둘 다 적용

#### 4. CNN

In [33]:
class ImageCNN(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(CustomModel, self).__init__()
        self.cnn_extract = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2, stride=2), 
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def forward(self, img):
        img_feature = self.cnn_extract(img) # cnn_extract 적용
        img_feature = torch.flatten(img_feature, start_dim=1) # 1차원으로 변환
        return output

#### input_channel = 3: RGB 3개의 채널이기 때문
#### out_channel = 8: 출력하는 채널 8개
#### kernel_size = 3: convolution의 시야를 결정. 보통 2d에서 3x3 픽셀로 사용합니다.
#### stride = 1: 이미지를 횡단할 떄 커널의 스텝 사이즈를 결정. 기본값은 1이지만 보통 max pooling과 비슷하게 이미지를 다운샘플링하기 위해 stride를 2로 사용할 수 있습니다.
#### ReLu() = Rectified Linear Unit의 약자로 입력이 0이하면 0으로 침묵, 0을 넘으면 입력 그대로를 출력하는 함수